In [3]:
using Rocket
using BenchmarkTools

In [4]:
plusone(d) = d + one(d)

function loop_draft()
    m_ab = lazy(Int)
    m_ac = lazy(Int)

    m_ba = lazy(Int)
    m_bc = lazy(Int)

    m_ca = lazy(Int)
    m_cb = lazy(Int)
    
    q1 = BehaviorSubject(0)
    q2 = BehaviorSubject(0)
    q3 = BehaviorSubject(0)
    
    q_algorithm = discontinue() + map(Int, sum) + share()

    q_up_1 = combineLatest(m_ac, m_ca) |> q_algorithm
    q_up_2 = combineLatest(m_bc, m_cb) |> q_algorithm
    q_up_3 = combineLatest(m_ab, m_ba) |> q_algorithm

    subscribe!(q_up_1, q1)
    subscribe!(q_up_2, q2)
    subscribe!(q_up_3, q3)
    
#     s1 = subscribe!(q1, logger("q1"))
#     s2 = subscribe!(q2, logger("q2"))
#     s3 = subscribe!(q3, logger("q3"))
    
    m_algorithm = map(Int, plusone)
    
    set!(m_ab, q1 |> m_algorithm)
    set!(m_ba, q2 |> m_algorithm)
    set!(m_bc, q3 |> m_algorithm)
    set!(m_ba, q2 |> m_algorithm)
    set!(m_ca, q2 |> m_algorithm)
    set!(m_cb, q1 |> m_algorithm)
    
#     sa1 = subscribe!(m_ab |> skip_complete(), logger("m_ab"))
#     sa2 = subscribe!(m_ba |> skip_complete(), logger("m_ba"))
#     sb1 = subscribe!(m_bc |> skip_complete(), logger("m_bc"))
#     sb2 = subscribe!(m_ba |> skip_complete(), logger("m_ba"))
#     sc1 = subscribe!(m_ca |> skip_complete(), logger("m_ca"))
#     sc2 = subscribe!(m_cb |> skip_complete(), logger("m_cb"))
    
    next!(q1, q1.props.current + 1)
    next!(q2, q2.props.current + 1)
    next!(q3, q3.props.current + 1)
    
    return (q1.props.current, q2.props.current, q3.props.current)
end

loop_draft (generic function with 1 method)

In [6]:
@btime loop_draft()

  10.850 μs (256 allocations: 10.45 KiB)


(1, 18, 21)

In [5]:
@time loop_draft()

  0.661958 seconds (2.17 M allocations: 111.753 MiB, 3.28% gc time)


(1, 18, 21)